In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
import pygplates
import os
from IPython.display import Image
from resolve_topologies import resolve_topologies as topology2gmt
%matplotlib inline


In [16]:
input_feature_filename = '/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/Coastlines/Coastlines_low_res_Matthews++.shp'
input_rotation_filename = ['/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/Global_EB_250-0Ma_GK07_Matthews++.rot',
                           '/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/Global_EB_410-250Ma_GK07_Matthews++.rot']
#input_rotation_filename = '/Users/wenchaocao/Desktop/New_Rotation.rot'

Data = pd.read_csv('/Users/wenchaocao/Research/5_ConsistencyRatios_Fossils_Paleogeog/ReconPaleogeog_EB2015v1_AfterModifiedBasedOnFossils_410_2Ma/ReconPaleogeog_EB2015v1_AfterModifiedOnFossils_TestByFossils_410_2Ma.csv', delimiter=',')  
reconstruction_time_list = Data['Time intervals']
FigNum_list = np.arange(64,17,-2)
FromAge_list = Data['From_age']
ToAge_list = Data['To_age']

reconstruction_time = []
FigNum = []
FromAge= []
ToAge= []
anchor_plate= 0

for i,reconstruction_time,FigNum,FromAge,ToAge in zip(np.arange(1,2,1),reconstruction_time_list,FigNum_list,FromAge_list,ToAge_list):
    print reconstruction_time,FigNum,FromAge, ToAge

    # Name of ouput file
    output_reconstructed_feature_filename = 'coastlines_%0.2fMa.shp' % reconstruction_time

    # Use pygplates to carry out the reconstruction 
    pygplates.reconstruct(input_feature_filename, input_rotation_filename, output_reconstructed_feature_filename, reconstruction_time, anchor_plate) 

    #print output_reconstructed_feature_filename
    input_topology_filename = ['/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/Mesozoic-Cenozoic_plate_boundaries_Matthews++.gpml', 
                              '/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/Paleozoic_plate_boundaries_Matthews++.gpml',
                              '/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/TopologyBuildingBlocks_Matthews++.gpml']

    output_filename_prefix = ''
    output_filename_extension = 'gmt'

    topology2gmt(input_rotation_filename,
                 input_topology_filename,
                 reconstruction_time,
                 output_filename_prefix,
                 output_filename_extension,
                 0)

    outfile='/Users/wenchaocao/Desktop/Temp/Temp_GeoTiff/PaleogeogRecon_Matthews2016_'+str(reconstruction_time)+'Ma.ps'

    #basedir = '/Users/wenchaocao/Research/6_CookieCutting_Corrections/'+str(reconstruction_time)+'/TopologyMethod_'+str(reconstruction_time)+'Ma/8_Reconstructed_Paleogeog_EBIDs_'+str(reconstruction_time)+'Ma_Modified_Paleobiology'
    basedir = '/Users/wenchaocao/Research/6_CookieCutting_Corrections/'+str(reconstruction_time)+'/15_Reconstructed_Paleogeog_Matthews2016_'+str(reconstruction_time)+'Ma_Corrected'

    #print basedir
    icesheet =  '%s/i_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (basedir,FigNum,FromAge,ToAge,reconstruction_time)
    mountain =  '%s/m_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (basedir,FigNum,FromAge,ToAge,reconstruction_time)
    landmass = '%s/lm_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (basedir,FigNum,FromAge,ToAge,reconstruction_time)
    shallow_marine = '%s/sm_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (basedir,FigNum,FromAge,ToAge,reconstruction_time)

    # Select a projection - central meridian is set to 30 deg longitude for each
    #proj='-Ji30/0.02i'  # uncomment for Sinusoidal projection
    #proj='-JN30/5i'      # uncomment for Robinson projection
    #proj='-JW0/10i'    # uncomment for Mollweide projection  proj='-JW30/10i'
    proj='-JQ0/0/10i'  #-JQ[lon0/[lat0/]]width (Cylindrical Equidistant)

    #os.system('gmt gmtset FONT_ANNOT_PRIMARY +20p')
    os.system('gmt gmtset MAP_FRAME_PEN 0c,white') #0.02c white
    os.system('gmt gmtset MAP_TICK_LENGTH_PRIMARY 0p/0p') #4p/2p
    os.system('gmt gmtset MAP_FRAME_TYPE inside') # set ticks inside the frame
    #os.system('gmt gmtset PS_PAGE_COLOR green')
    #os.system('gmt gmtset COLOR_FOREGROUND yellow')
    
    # None #-G79/148/205 ,70/130/180 ,0/104/139
    os.system('gmt psbasemap -Rg -BN -G0/104/139 %s -K > %s ' % (proj,outfile)) # -Ba30f30we -Gghostwhite  -Rg
    ###############################################################

    #rename files (shapefile -> shapefile; shapefile ->.gmt)
    tmp = pygplates.FeatureCollection(shallow_marine)
    tmp.write('sm.gmt')
    tmp = pygplates.FeatureCollection(landmass)
    tmp.write('lm.gmt')
    tmp = pygplates.FeatureCollection(mountain)
    tmp.write('m.gmt')

    if os.path.isfile(icesheet):
        tmp = pygplates.FeatureCollection(icesheet)
        tmp.write('i.gmt')
    else:
            print 'No icesheet found' 

    tmp = pygplates.FeatureCollection(output_reconstructed_feature_filename)
    tmp.write('coastlines.gmt')

    # clip the extra geometries but it has to be used twice.
    os.system('gmt psclip clip.txt -Rg %s -O -V -K >> %s' % (proj,outfile))
    
    #plot paleogeographic features
    os.system('gmt psxy -R %s -W0p,lightblue -Glightblue sm.gmt -K -O -N >> %s' % (proj,outfile)) #0.1
    os.system('gmt psxy -R %s -W0p,yellow -Gyellow lm.gmt -K -O -N >> %s' % (proj,outfile))
    os.system('gmt psxy -R %s -W0p,orange -Gorange m.gmt -K -O -N >> %s' % (proj,outfile))
    if os.path.isfile(icesheet):
        os.system('gmt psxy -R %s -W0p,223/223/255 -G223/223/255 i.gmt -K -O -N >> %s' % (proj,outfile))
    else:
        print 'No icesheet found'

    #plot coastlines
    os.system('gmt psxy -R %s -W0.2p,grey60 -K -O -m coastlines.gmt -V >> %s' % (proj,outfile)) #0.2 
    
    #close the psclip here
    os.system('gmt psclip clip.txt -Rg %s -O -V -C >> %s' % (proj,outfile))

    #convert ps into raster, -E set the resolution
    #b means BMP, e means EPS, E means EPS with PageSize command, f means PDF, F means multi-page PDF, j means JPEG, g means PNG, G means transparent PNG (untouched regions are transparent), m means PPM, and t means TIFF [default is JPEG]
    os.system('gmt ps2raster %s -A -E600 -Tg -P' % outfile)  #-F${reconstruction_time}Ma
    os.system('gmt ps2raster %s -A -E600 -Tf -P' % outfile)  #-F${reconstruction_time}Ma
    os.system('gmt ps2raster %s -A -E600 -Tj -P' % outfile)
    os.system('gmt ps2raster %s -E600 -Tt -A -W+g -P -V ' % outfile)
    
    # -co "COMPRESS=method": LZW - highest compression ratio, highest processing power
    #                        DEFLATE
    #                        PACKBITS - lowest compression ratio, lowest processing power
    # With lzw and deflate compression using -co predictor=2 can help with imagery that is smoothly varying as it compresses the differences from pixel to pixel instead of the absolute values, and these will tend to be small and have more patterns (ref). Predictor is only useful with lzw and deflate compression, the option has no effect with other methods. 
    os.system('gdal_translate -co compress=lzw -co predictor=2 %s PaleogeogRecon_Matthews2016_%sMa.tiff -a_srs WGS84' % (outfile.replace('.ps', '.tiff'),reconstruction_time)) # georeference the file again
    
    #os.system('convert -resize 6002 %s PaleogeogRecon_Matthews2016_Resized_%s.tiff ' % (outfile.replace('.ps', '.tiff'),reconstruction_time))
    #Image('/Users/wenchaocao/Research/21_Python/Output_Temp/Paleogeog_Reconstructed_Matthews2016_'+str(reconstruction_time)+'.png')

6 64 11 2
